In [6]:
from codice import mymodel
from codice.mymodel import modello_albero
import pyomo.environ as pyo
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff 
from sklearn.model_selection import train_test_split
from codice.mytree import Tree
import importlib
from codice import mynode 
importlib.reload(mymodel)
importlib.reload(mynode)

SEED = 1

### PREPROCESS DATASET ###

# TODO: NORMALIZZARE DATI
raw_data = loadarff('../dataset/Rice_Cammeo_Osmancik.arff')
df_data = pd.DataFrame(raw_data[0]).to_numpy()

# TODO FIXARE ! SONO TUTTI UNI
x_data_raw = df_data[:, :-1]
# Cammeo = 0, Osmancik = 1
y_data_raw = [0 if x.decode("utf-8") == "Cammeo" else 1 for x in df_data[:,-1]]

X_train, X_test, y_train, y_test = train_test_split(x_data_raw, y_data_raw, test_size=0.95, stratify=y_data_raw, 
                                                        random_state=SEED) 

num_features = len(X_train[0])


-> Branch1
    -> Branch2
        -> Leaf4
        -> Leaf5
    -> Branch3
        -> Leaf6
        -> Leaf7


In [7]:
### BUILD PARAMETERS ###
alpha = 0.5 # complexity
beta = 5 # n min of elements in a leaf
delta = 2 # depth

# compute epsilon value (uj)
u = [None for _ in range(num_features)]
for j in range(num_features):
    for i in range(len(X_train)-1):
        diff = abs(X_train[i][j] - X_train[i+1][j])
        if (u[j] == None or diff < u[j]) and diff != 0:
            u[j] = diff
umax = max(u)
umin = min(u)

## TODO: capire come calcolarlo
lcap = 0.5 

yi_class = [[None for _ in range(len(np.unique(y_train)))] for _ in range(len(X_train))]
for i in range(len(X_train)):
    for col in range(len(np.unique(y_train))):
        yi_class[i][col] = 1 if y_train[i] == col else 0


mvalue = len(X_train)

print(u)
print(umin)
print(umax)



### BUILD TREE ###
tree = Tree(delta)

print(tree.get_parent(4))

[28.0, 0.128021240234375, 0.1252288818359375, 0.060638427734375, 2.8967857360839844e-05, 43.0, 0.0008328557014465332]
2.8967857360839844e-05
43.0
2


In [8]:
import importlib
import time
importlib.reload(mymodel)

tstart = time.time()
modello = mymodel.modello_albero(alpha, beta, delta, u, umax, umin, num_features, X_train, y_train, tree, lcap, yi_class, mvalue)
opt = pyo.SolverFactory('gurobi')
results = opt.solve(modello, tee=True) # solves and updates model
pyo.assert_optimal_termination(results)
tend = time.time() - tstart

7
[[2, 1], [1], [3], []]
[None, None, None]
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-20
Read LP format model from file C:\Users\march\AppData\Local\Temp\tmp3cjjm7v9.pyomo.lp
Reading time = 0.03 seconds
x1: 2517 rows, 815 columns, 18369 nonzeros
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 6900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2517 rows, 815 columns and 18369 nonzeros
Model fingerprint: 0xee45fd6d
Variable types: 3 continuous, 812 integer (796 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+04]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 9 rows and 21 columns
Presolve time: 0.03s
Presolved: 2508 rows, 794 columns, 9225 nonzeros
Variable types: 3 continuous, 791 integer (7

In [11]:
amatrix = np.empty((num_features, 2**delta-1))
for riga in range(num_features):
    for col in range(1, 2**delta-1):
        amatrix[riga, col] = modello.a[riga, col].value

cvector = []
for col in range(2**delta, 2**(delta+1)):
    if modello.c[0, col].value == 1:
        cvector.append(0)
    else:
        cvector.append(1)

In [13]:
from codice import mynode
importlib.reload(mynode)

validationtree = mynode.create_complete_tree(1,depth=delta)
mynode.init_bfs(validationtree, avalues=amatrix, bvalues=modello.b, dvalues=modello.d, classnames=cvector, depth=delta)

numcorretti = 0
for i in range(len(y_test)):
    numcorretti += mynode.validate(X_test[i, :], y_test[i], validationtree)

print(numcorretti)
print(len(y_test))
print("Accuracy: ", (numcorretti/len(y_test))*100, "%")

-> Branch1
    -> Branch2
        -> Leaf4
        -> Leaf5
    -> Branch3
        -> Leaf6
        -> Leaf7
1549
3620
Accuracy:  42.790055248618785 %
